In [ ]:
###r2---for MLMS_MSE

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from torch_geometric.loader import DataLoader
import torch.nn as nn
from torch_geometric.nn import GCNConv,EdgeConv
from torch_geometric.nn import global_mean_pool,global_max_pool
from torch.utils.data import random_split
import torch.nn.functional as F
from tqdm import trange

In [ ]:
hidden_channels = 512
cos_ratio = 0
mse_ratio = 1
lr = 0.0001
class GNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        torch.manual_seed(42)
       
        self.conv1 = GCNConv(45, hidden_channels)
        self.conv2 = GCNConv(hidden_channels,hidden_channels)
        self.conv3 = GCNConv(hidden_channels,hidden_channels)
        self.conv4 = GCNConv(hidden_channels,hidden_channels)
        self.conv5 = GCNConv(hidden_channels,hidden_channels)
        
        self.lin1 = nn.Linear(2*hidden_channels,hidden_channels)
        self.lin2 = nn.Linear(hidden_channels,hidden_channels)
        self.out = nn.Linear(hidden_channels, 50)
        
        self.loss_function1 = nn.CosineEmbeddingLoss(margin=0)
        self.loss_function2 = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=lr, weight_decay=5e-4)
        self.counter=0
        self.progress = []
        
    def forward(self, x, edge_index, batch):
        
        x = x.to(device)
        edge_index = edge_index.to(device)
        batch = batch.to(device)
        
        x=self.conv1(x, edge_index)
        x = x.relu()
        x=self.conv2(x, edge_index)
        x = x.relu()
        x=self.conv3(x, edge_index)
        x = x.relu()
        x=self.conv4(x, edge_index)
        x = x.relu()
        x=self.conv5(x, edge_index)
        x = x.relu()
        
        x = torch.cat((global_mean_pool(x,batch),global_max_pool(x,batch)),dim=1)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        x = x.relu()
        out = self.out(x)
        return out
    
    def train(self,data):
        
        self.optimizer.zero_grad()
        
        outputs = self.forward(data.x.float(),data.edge_index,data.batch)
        y = data.y.to(device)
        batch_size = data.batch_size
        target = torch.ones(batch_size)
        target = target.to(device)
        loss = cos_ratio*self.loss_function1(outputs.to(torch.float32),y.view(batch_size,50).to(torch.float32),target)+mse_ratio*self.loss_function2(outputs.to(torch.float32),y.view(batch_size,50).to(torch.float32))
        
        self.progress.append(loss.item())        
        loss.backward()
        self.optimizer.step()
        
        return loss
        
    def test(self,data):
        outputs = self.forward(data.x.float(),data.edge_index,data.batch)
        target = torch.ones(len(test_data))
        target = target.to(device)
        y = data.y.to(device)
        acc = cos_ratio*self.loss_function1(outputs.to(torch.float32),y.view(len(test_data),50).to(torch.float32),target)+mse_ratio*self.loss_function2(outputs.to(torch.float32),y.view(len(test_data),50).to(torch.float32))
        return acc
    
    def pred(self,data):
        outputs = self.forward(data.x.float(),data.edge_index,data.batch)
        return outputs
    

In [ ]:
device = torch.device('cuda:4' if torch.cuda.is_available() else 'cpu')
model = GNN()
model.load_state_dict(torch.load("/home/chengc/workspace/cc/0815/model/qm_9_mse_model.pth"))
model.to(device)

In [ ]:
raw_data = torch.load("/home/chengc/workspace/cc/new_exp_0427/data/qm9_ir_broaden/qm9_input.pt")
num_train = int(len(raw_data)*0.8)
num_val = int(len(raw_data)-num_train)
train_data,test_data = random_split(dataset = raw_data,lengths = [num_train,num_val],generator = torch.Generator().manual_seed(42))
batch_size = 64
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(test_data, batch_size = len(test_data))

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
result = model.pred(next(iter(test_loader)))

In [ ]:
real = next(iter(test_loader)).y

In [ ]:
y_true = real[0:50]
y_pred = result[0].detach().cpu()
r2 = r2_score(y_true, y_pred)  
print(f'R^2: {r2}')

In [ ]:
r2_list = []
for i in trange(len(result)):
    y_true = real[(0+i*50):(50+i*50)]
    y_pred = result[i].detach().cpu()
    r2 = r2_score(y_true, y_pred)
    r2_list.append(r2)

In [ ]:
np.mean(r2_list)###1000轮 0.8375630424836698

In [ ]:
### r2 for MLMS_2IN1

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from torch_geometric.loader import DataLoader
import torch.nn as nn
from torch.utils.data import random_split
from tqdm import trange
from sklearn.metrics import r2_score

In [2]:
hidden_channels = 256
class twoinone(nn.Module):
    
    def __init__(self):
        super().__init__()
        torch.manual_seed(42)
       
        self.nn1 = nn.Linear(100, hidden_channels*2)
        self.nn2 = nn.Linear(hidden_channels*2,hidden_channels*2)
        self.nn3 = nn.Linear(hidden_channels*2,hidden_channels*2)
        self.nn4 = nn.Linear(hidden_channels*2,hidden_channels*2)
        self.out = nn.Linear(hidden_channels*2, 50)
        
        self.loss_function = nn.MSELoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.0001, weight_decay=5e-4)
        self.counter=0
        self.progress = []
        
    def forward(self,x):
        
        x = x.to(device)
        x=self.nn1(x)
        x = x.relu()
        x=self.nn2(x)
        x = x.relu()
        x=self.nn3(x)
        x = x.relu()
        x=self.nn4(x)
        x = x.relu()
        out = self.out(x)
        
        return out
    
    def train(self,dataloader):
        losses=[]
        for i in range(len(dataloader)):
            data = dataloader[i]
            outputs = self.forward(data.x.float())
            y = data.y.to(device)
            y = y.float()
            loss_t = self.loss_function(outputs.to(torch.float32),y.to(torch.float32))
            losses.append(loss_t)
        loss = torch.stack(losses,dim=0).mean(dim=0)        
        #outputs = self.forward(data.x.float())
        #y = data.y.to(device)
        #y = y.float()
        #loss = self.loss_function(outputs.to(torch.float32),y.to(torch.float32))
       
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss
    
    def val(self,dataloader):
        losses=[]
        for i in range(len(dataloader)):
            data = dataloader[i]
            outputs = self.forward(data.x.float())
            y = data.y.to(device)
            y = y.float()
            loss_t = self.loss_function(outputs.to(torch.float32),y.to(torch.float32))
            losses.append(loss_t)
        loss = torch.stack(losses,dim=0).mean(dim=0)
        return loss
    
    def pred(self,data):
        output = self.forward(data.x.float())
        return output

In [3]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = twoinone()
model.load_state_dict(torch.load("/home/chengc/workspace/cc/0815/model/qm_9_2in1_model.pth"))
model.to(device)

twoinone(
  (nn1): Linear(in_features=100, out_features=512, bias=True)
  (nn2): Linear(in_features=512, out_features=512, bias=True)
  (nn3): Linear(in_features=512, out_features=512, bias=True)
  (nn4): Linear(in_features=512, out_features=512, bias=True)
  (out): Linear(in_features=512, out_features=50, bias=True)
  (loss_function): MSELoss()
)

In [4]:
data = torch.load("/home/chengc/workspace/cc/0815/qm_9_2in1.pt")
train_data,val_data,test_data = random_split(dataset = data,lengths = [100000,20000,7468],generator = torch.Generator().manual_seed(42))

In [7]:
#test_r2
r2_list = []
for i in trange(len(test_data)):
    y_true = test_data[i].y
    y_pred = model.pred(test_data[i]).detach().cpu()
    r2 = r2_score(y_true, y_pred) 
    r2_list.append(r2)
np.mean(r2_list)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7468/7468 [00:10<00:00, 718.52it/s]


0.8469676325560166

In [8]:
#val_r2
r2_list = []
for i in trange(len(val_data)):
    y_true = val_data[i].y
    y_pred = model.pred(val_data[i]).detach().cpu()
    r2 = r2_score(y_true, y_pred) 
    r2_list.append(r2)
np.mean(r2_list)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:25<00:00, 799.38it/s]


0.8513208359148543

In [9]:
#train_r2
r2_list = []
for i in trange(len(train_data)):
    y_true = train_data[i].y
    y_pred = model.pred(train_data[i]).detach().cpu()
    r2 = r2_score(y_true, y_pred) 
    r2_list.append(r2)
np.mean(r2_list)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100000/100000 [02:08<00:00, 777.15it/s]


0.9171942400406787